In [20]:
# stdlib
import os
import time

# third party
import docker
import numpy as np

# syft absolute
import syft as sy

# syft absolute
from syft.service.worker.image_registry import SyftImageRegistry
from syft.service.worker.worker_image import SyftWorkerImage

print(sy.__version__)

0.9.1-beta.4


In [2]:

"""
1. Launch cluster:
CLUSTER_NAME=test-datasite-1 CLUSTER_HTTP_PORT=8080 tox -e dev.k8s.start
CLUSTER_NAME=test-datasite-1 CLUSTER_HTTP_PORT=8080 tox -e dev.k8s.deploy
"""

os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
os.environ["DEV_MODE"] = "True"
os.environ["SERVER_PORT"] = "8080"

In [3]:
datasite = sy.orchestra.launch(
    name="test-datasite-1",
    dev_mode=True,
    reset=True,
    create_producer=True,
    n_consumers=2,
    port="auto",
)

In [6]:
datasite_client = datasite.login(email="info@openmined.org", password="changethis")

Logged into <syft-dev-server: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

## Setup custom pool

In [21]:
registry_url = "localhost:5800"

datasite_client.api.services.image_registry.add(registry_url)
registry = datasite_client.api.services.image_registry[0]

In [7]:
syft_base_worker_tag = "0.9.0-beta.5"

custom_dockerfile_str = f"""
FROM openmined/syft-backend:{syft_base_worker_tag}

RUN uv pip install recordlinkage
""".strip()

docker_config = sy.DockerWorkerConfig(dockerfile=custom_dockerfile_str)
docker_tag = "openmined/custom-worker-recordlinkage:latest"

In [11]:
submit_result = datasite_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [26]:
workerimage = datasite_client.images.get_all()[1]
workerimage

```python
class SyftWorkerImage:
  id: str = ec68c829a44e4182b213c6f1f84dd3f1
  image_identifier: str = None
  image_hash: str = None
  created_at: str = 2024-08-15 09:46:40
  built_at: str = None
  config: str = FROM openmined/syft-backend:0.9.0-beta.5

RUN uv pip install recordlinkage

```

In [28]:
docker_build_result = datasite_client.api.services.worker_image.build(
    image_uid=workerimage.id,
    tag=docker_tag,
    registry_uid=registry.id,
    pull_image=True,
)
docker_build_result

SyftSuccess: Build for Worker ID: ec68c829a44e4182b213c6f1f84dd3f1 succeeded.
----------Logs for pod=build-db87625634082ef8-msz4d----------
time="2024-08-15T10:02:15Z" level=info msg="Retrieving image manifest openmined/syft-backend:0.9.0-beta.5"time="2024-08-15T10:02:15Z" level=info msg="Retrieving image openmined/syft-backend:0.9.0-beta.5 from registry index.docker.io"time="2024-08-15T10:02:16Z" level=info msg="Retrieving image manifest openmined/syft-backend:0.9.0-beta.5"time="2024-08-15T10:02:16Z" level=info msg="Returning cached image manifest"time="2024-08-15T10:02:16Z" level=info msg="Built cross stage deps: map[]"time="2024-08-15T10:02:16Z" level=info msg="Retrieving image manifest openmined/syft-backend:0.9.0-beta.5"time="2024-08-15T10:02:16Z" level=info msg="Returning cached image manifest"time="2024-08-15T10:02:16Z" level=info msg="Retrieving image manifest openmined/syft-backend:0.9.0-beta.5"time="2024-08-15T10:02:16Z" level=info msg="Returning cached image manifest"time="2024-08-15T10:02:16Z" level=info msg="Executing 0 build triggers"time="2024-08-15T10:02:16Z" level=info msg="Building stage 'openmined/syft-backend:0.9.0-beta.5' [idx: '0', base-idx: '-1']"time="2024-08-15T10:02:16Z" level=info msg="Checking for cached layer registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage/cache:24d1d55bd983c4b76d13e0d05393238eb0ec4a88e1f60dd0f3ced8a5e8a07eaf..."time="2024-08-15T10:02:17Z" level=info msg="No cached layer found for cmd RUN uv pip install recordlinkage"time="2024-08-15T10:02:17Z" level=info msg="Unpacking rootfs as cmd RUN uv pip install recordlinkage requires it."time="2024-08-15T10:02:36Z" level=info msg="RUN uv pip install recordlinkage"time="2024-08-15T10:02:36Z" level=info msg="Initializing snapshotter ..."time="2024-08-15T10:02:36Z" level=info msg="Taking snapshot of full filesystem..."time="2024-08-15T10:02:38Z" level=info msg="Cmd: /bin/sh"time="2024-08-15T10:02:38Z" level=info msg="Args: [-c uv pip install recordlinkage]"time="2024-08-15T10:02:38Z" level=info msg="Util.Lookup returned: &{Uid:0 Gid:0 Username:root Name: HomeDir:/root}"time="2024-08-15T10:02:38Z" level=info msg="Performing slow lookup of group ids for root"time="2024-08-15T10:02:38Z" level=info msg="Running: [/bin/sh -c uv pip install recordlinkage]"Audited 1 package in 59mstime="2024-08-15T10:02:38Z" level=info msg="Taking snapshot of full filesystem..."time="2024-08-15T10:02:39Z" level=info msg="Pushing layer registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage/cache:24d1d55bd983c4b76d13e0d05393238eb0ec4a88e1f60dd0f3ced8a5e8a07eaf to cache now"time="2024-08-15T10:02:39Z" level=info msg="Pushing image to registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage/cache:24d1d55bd983c4b76d13e0d05393238eb0ec4a88e1f60dd0f3ced8a5e8a07eaf"time="2024-08-15T10:02:39Z" level=info msg="Pushed registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage/cache@sha256:58aabdc17507169b6eca16e950cde26703d7c18eea8fe0a4b048c3de1561671f"time="2024-08-15T10:02:39Z" level=info msg="Pushing image to registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage:latest"time="2024-08-15T10:02:52Z" level=info msg="Pushed registry.syft.svc.cluster.local/openmined/custom-worker-recordlinkage@sha256:46c860184e97a6e50ba6bba344a7d71a836ae2347e6a438608e66abbd609e494"

In [30]:
workerimage = datasite_client.images[1]
workerimage

```python
class SyftWorkerImage:
  id: str = ec68c829a44e4182b213c6f1f84dd3f1
  image_identifier: str = localhost:5800/openmined/custom-worker-recordlinkage:latest
  image_hash: str = "sha256:46c860184e97a6e50ba6bba344a7d71a836ae2347e6a438608e66abbd609e494"
  created_at: str = 2024-08-15 09:46:40
  built_at: str = 2024-08-15 10:02:55
  config: str = FROM openmined/syft-backend:0.9.0-beta.5

RUN uv pip install recordlinkage

```

In [31]:
pool_name = "my-pool"
worker_pool_res = datasite_client.api.services.worker_pool.launch(
    pool_name=pool_name,
    image_uid=workerimage.id,
    num_workers=2,
)
worker_pool_res

SyftError: Failed to start workers  <class 'TimeoutError'> () Traceback (most recent call last):
  File "/root/app/.venv/lib/python3.12/site-packages/anyio/_core/_tasks.py", line 115, in fail_after
    yield cancel_scope
  File "/root/app/.venv/lib/python3.12/site-packages/kr8s/_objects.py", line 474, in wait
    async for _ in self.async_watch():
  File "/root/app/.venv/lib/python3.12/site-packages/kr8s/_objects.py", line 351, in async_watch
    async for event, obj in self.api.async_watch(
  File "/root/app/.venv/lib/python3.12/site-packages/kr8s/_api.py", line 430, in async_watch
    async for line in response.aiter_lines():
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_models.py", line 963, in aiter_lines
    async for text in self.aiter_text():
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_models.py", line 950, in aiter_text
    async for byte_content in self.aiter_bytes():
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_models.py", line 929, in aiter_bytes
    async for raw_bytes in self.aiter_raw():
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_models.py", line 987, in aiter_raw
    async for raw_stream_bytes in self.stream:
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_client.py", line 149, in __aiter__
    async for chunk in self._stream:
  File "/root/app/.venv/lib/python3.12/site-packages/httpx/_transports/default.py", line 254, in __aiter__
    async for part in self._httpcore_stream:
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 367, in __aiter__
    raise exc from None
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/connection_pool.py", line 363, in __aiter__
    async for part in self._stream:
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/http11.py", line 349, in __aiter__
    raise exc
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/http11.py", line 341, in __aiter__
    async for chunk in self._connection._receive_response_body(**kwargs):
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/http11.py", line 210, in _receive_response_body
    event = await self._receive_event(timeout=timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_async/http11.py", line 224, in _receive_event
    data = await self._network_stream.read(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/app/.venv/lib/python3.12/site-packages/httpcore/_backends/anyio.py", line 35, in read
    return await self._stream.receive(max_bytes=max_bytes)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/app/.venv/lib/python3.12/site-packages/anyio/streams/tls.py", line 205, in receive
    data = await self._call_sslobject_method(self._ssl_object.read, max_bytes)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/app/.venv/lib/python3.12/site-packages/anyio/streams/tls.py", line 147, in _call_sslobject_method
    data = await self.transport_stream.receive()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/app/.venv/lib/python3.12/site-packages/anyio/_backends/_asyncio.py", line 1142, in receive
    await self._protocol.read_event.wait()
  File "/usr/lib/python3.12/asyncio/locks.py", line 212, in wait
    await fut
asyncio.exceptions.CancelledError: Cancelled by cancel scope ffff3a5cb620

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/app/syft/src/syft/service/worker/utils.py", line 357, in create_kubernetes_pool
    pool = runner.create_pool(
           ^^^^^^^^^^^^^^^^^^^
  File "/root/app/syft/src/syft/custom_worker/runner_k8s.py", line 63, in create_pool
    deployment.wait(
  File "/root/app/.venv/lib/python3.12/site-packages/kr8s/_io.py", line 75, in wrapped
    return portal.call(wrapped)
           ^^^^^^^^^^^^^^^^^^^^
  F

## Run job on pool

In [ ]:
dataset = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=np.array([10, 11, 12, 13, 14]),
            data=np.array([[15, 16, 17, 18, 19] for _ in range(100_000)]),
            mock_is_real=True,
        )
    ],
)

datasite_client.upload_dataset(dataset)

In [ ]:
data = datasite_client.datasets[0].assets[0]

@sy.syft_function_single_use(data=data, worker_pool_name=pool_name)
def compute_mean(datasite, data) -> float:
    print("Computing mean...")
    return data

In [ ]:
datasite_client.code.request_code_execution(compute_mean)

In [ ]:
job = datasite_client.code.compute_mean(data=data, blocking=False)
job.wait()

display(job)

## Prepare migration

In [33]:
from pathlib import Path

migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
migration_data_dir.mkdir(exist_ok=True)

blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

blob_path.unlink(missing_ok=True)
yaml_path.unlink(missing_ok=True)

In [35]:
migration_data = datasite_client.get_migration_data(include_blobs=True)

migration_data

```python
class MigrationData:
  id: str = f62cbcadbfcb42069174dc71e4dfff09
  server_uid: str = 913be68f34d248349421d6760cd69811
  root_verify_key: str = d65da05ec0ae1d1a3d7544cb3d3eadc9c439ef46848f7723ec91545cb3613270
  num_objects: str = 44
  num_action_objects: str = 0
  includes_blobs: str = True

```

In [36]:
migration_data.save(blob_path, yaml_path=yaml_path)

assert blob_path.exists()
assert yaml_path.exists()

print(f"Saved migration data to {str(blob_path.resolve())}")

Saved migration data to /Users/eelco/dev/PySyft/notebooks/tutorials/version-upgrades/migration.blob


## Reset node, add migration data

In [38]:
# datasite = sy.orchestra.launch(
#     name="test-datasite-1",
#     dev_mode=True,
#     reset=True,
#     create_producer=True,
#     n_consumers=2,
#     port="auto",
# )
# datasite_client = datasite.login(email="info@openmined.org", password="changethis")

In [41]:
assert(len(datasite_client.images.get_all()) == 1), "This node has not been reset"

AssertionError: This node has not been reset

In [42]:
datasite_client.load_migration_data(blob_path)

SyftSuccess: Migration completed successfully